In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import time
import datetime
from copy import deepcopy as copy

In [8]:
plt.style.use("ggplot")
plt.tight_layout()
plt.ioff()
plt.rcParams["figure.figsize"] = (16,8)

In [9]:
%run cards.ipynb
%run game.ipynb
%run visualization.ipynb
d = Visualization()